In [1]:
import pandas as pd
import os
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.font_manager as fm
import shap
shap.initjs()
from shap.maskers import Independent

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


load data

In [2]:
# load fc_feature
feature_table = pd.read_csv('./data/fc_feature.csv')
feature = np.array(feature_table)

In [4]:
# original node_node
columns = list(feature_table.columns)

node

In [ ]:
# Five model node_nodes
node_link_5model = pd.read_csv('./node/node_link_5models.csv')
node_node = list(node_link_5model['Unnamed: 0'])
# len(node_node)

In [5]:
# Find the index of the corresponding node_node in the original columns
node_index = []
for i in range(len(node_node)):
    index = columns.index(node_node[i])
    node_index.append(index)

mean_shap matrices for 5 models

In [6]:
models = ['SVM', 'LR', 'RF', 'XGBoost', 'AdaBoost']

In [7]:
shap_svm = pd.read_csv('./mean_shap_values/mean_shap_values_svc.csv')
shap_lr = pd.read_csv('./mean_shap_values/mean_shap_values_lr.csv')
shap_rf = pd.read_csv('./mean_shap_values/mean_shap_values_rf.csv')
shap_xg = pd.read_csv('./mean_shap_values/mean_shap_values_xg.csv')
shap_ada = pd.read_csv('./mean_shap_values/mean_shap_values_ada.csv')
csv_files = [shap_svm, shap_lr, shap_rf, shap_xg, shap_ada]

In [8]:
shap_5models = pd.read_csv('./mean_shap_values/mean_shap_values_5models.csv')

Find the corresponding abbr(network)

In [14]:
def num_to_roi(node_node, aal90_network):
    df_node_node = pd.DataFrame(columns=['num_num', 'roi_roi'])
    df_node_node['num_num'] = node_node
    for i in range(len(df_node_node)):
        a = int(df_node_node['num_num'].values[i].split('_')[0])
        b = int(df_node_node['num_num'].values[i].split('_')[1])
        a_index = aal90_network[aal90_network['num']==a].index.tolist()[0]
        b_index = aal90_network[aal90_network['num']==b].index.tolist()[0]
        df_node_node['roi_roi'].values[i] = aal90_network['abbr_net'].values[a_index] + '_' + aal90_network['abbr_net'].values[b_index]
        roi_roi = df_node_node['roi_roi'].tolist()
    return df_node_node, roi_roi

In [15]:
aal90_network = pd.read_csv('./network/aal90_network.csv')

In [16]:
df_node_node, roi_roi = num_to_roi(node_node, aal90_network)

In [17]:
# Change the header to "abbr(network)"
df_columns, columns_roi_roi = num_to_roi(columns, aal90_network)
feature_table.columns = list(df_columns['roi_roi'].values)

###### plot_shap

In [9]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    code from https://matplotlib.org/stable/gallery/images_contours_and_fields/image_annotated_heatmap.html
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)


    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, shrink=0.75, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom", fontsize=12)

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar

In [10]:
def get_common_top_N(heatmap, node_index, columns):
    # Fetch the name and column data corresponding to the final node_node from the five models
    new_columns = []
    for i in node_index:
        new_columns.append(columns[i])
    new = []
    for i in node_index:
        new.append(heatmap[:, i])
    new_heatmap = np.array([list(np.concatenate(new, axis=0))])
    
    return new_heatmap, new_columns

In [33]:
def plot_shap_heatmap(models, csv_files, node_index, columns):
    from matplotlib import rc, rcParams
    rc('axes', linewidth=2)
    rcParams.update({'font.size': 10})
    
    heatmaps = [[] for _ in models]
    
    for i in range(len(models)):
        mean_abs, std_abs, columns_mean = shap_stat(csv_files[i])
        heatmaps[i].append(mean_abs)
        print(np.shape(heatmaps[i]))
#     print(heatmaps)
        
    for i, model in enumerate(models):
        heatmaps[i] = np.array(heatmaps[i])
#         print(heatmaps[i])
        hm, feature_names = get_common_top_N(heatmaps[i], node_index, columns)
#         print(np.shape(hm))
#         print(hm)
        
#         for j in range(hm.shape[0]):
#             hm[j, :] = hm[j, :] / np.max(hm[j, :])
#         fig, ax = plt.subplots(figsize=(12, 6))
#         im, cbar = heatmap(hm, [model], feature_names, ax=ax, vmin=0, vmax=1,
#                            cmap="cool", cbarlabel="Relative Importance")
#         plt.savefig('/home/yanghuan/DOC_scripts/shap_ML_1000/shap_ML_fig/shap_heatmap_{}.pdf'.format(model), dpi=600, bbox_inches='tight')

In [12]:
# Calculate the shap mean value and standard deviation of each feature
def shap_stat(shap_values):
    columns = shap_values.columns
    mean_abs = []
    std_abs = []
    for i in range(shap_values.shape[1]):
        m = np.mean(abs(shap_values.to_numpy()[:, i]))
        s = np.std(abs(shap_values.to_numpy()[:, i]))
        mean_abs.append(m)
        std_abs.append(s)

    return mean_abs, std_abs, columns.to_list()

In [16]:
# Fetch top N
def get_top_N(mean, std, columns, N):
    pool = list(zip(mean, std, columns))
    pool.sort()
    return pool[-N:]

In [120]:
# shap bar plot
def plot_shap_bar(shap_values, feature, columns_roi_roi, top, model):
    from matplotlib import rc, rcParams
    rc('axes', linewidth=2)
    rc('font', size=15)

    mean_abs, std_abs, columns = shap_stat(shap_values)
    pool = get_top_N(mean_abs, std_abs, columns_roi_roi, top)
    
    fig, ax = plt.subplots(figsize=(6, 10))
#     plt.barh([a[2] for a in pool], [a[0] for a in pool], color='#75bbfd', capsize=5)
    shap.summary_plot(shap_values, feature, columns_roi_roi, plot_type="bar", max_display=top, color='#75bbfd', show=False) 
    fig = plt.gcf()
    fig.set_size_inches(6, 10)
    ax.set_xlabel('Mean(|SHAP|)', fontsize=15)
    ax.spines['top'].set_color('none')
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_smart_bounds(True)
    ax.spines['bottom'].set_smart_bounds(True)
    ax.spines['bottom'].set_position(('axes', 0.01))
    ax.tick_params(axis='both', which='major', labelsize=15)
    plt.savefig('./shap_bar_{}.pdf'.format(model), dpi=600, bbox_inches='tight')
#     plt.close()

In [134]:
def plot_beeswarm(shap_values, feature_data, max_display, model):
    from matplotlib import rc, rcParams
    rc('axes', linewidth=2)
    size = 40
    rc('font', size=size)
    size = 20
    fig, ax = plt.subplots()
    shap.summary_plot(shap_values.to_numpy(), feature_data, max_display=max_display, show=False)
    fig = plt.gcf()
    fig.set_size_inches(6, 10)
    cbar_ax = fig.axes[-1]
    cbar_ax.tick_params(labelsize=size)
    ax.tick_params(axis='both', which='major', labelsize=size)
    ax.set_xlabel('SHAP value', fontsize=size)
    fig.savefig('./shap_beeswarm_{}.pdf'.format(model), dpi=600, bbox_inches='tight')
#     plt.close()

In [167]:
def plot_beeswarm(shap_values, feature_data, max_display, model):
    from matplotlib import rc, rcParams
    size = 40
    rc('axes', linewidth=2)
    rc('font', size=size)

    fig, ax = plt.subplots()
    shap.summary_plot(shap_values.to_numpy(), feature_data, max_display=max_display, show=False)
#     fig = plt.gcf()
    fig.set_size_inches(6, 10)
    cbar_ax = fig.axes[-1]
    cbar_ax.tick_params(labelsize=size-20)
    ax.tick_params(axis='both', which='major', labelsize=size-20)
    ax.tick_params(axis='x', which='major', labelsize=size-25)
    
    ax.set_xlabel('SHAP value', fontsize=size-20)
    fig.savefig('./shap_beeswarm_{}.pdf'.format(model), dpi=600, bbox_inches='tight')

###### plot

In [160]:
plot_shap_bar(shap_svm, feature, columns_roi_roi, 17, 'svm')

In [161]:
plot_shap_bar(shap_5models, feature, columns_roi_roi, 19, '5models')

In [ ]:
plot_beeswarm(shap_svm, feature_table, 17, 'svm')

In [ ]:
plot_beeswarm(shap_5models, feature_table, 19, '5models')

In [ ]:
plot_shap_heatmap(models, csv_files, node_index, columns_roi_roi)